In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from math import exp, log, cos, cosh, sqrt, sinh, pi

In [3]:
from scipy.integrate import quad

In [8]:
@jit
def f(t_rn, h, T, nj):
    return exp(-h*cos(2*pi*t_rn/T)*nj)

In [21]:
h = 0.2
T = 1000
nj = 1



inte_1 = quad(f, 15, 17, args=(h, T, nj))[0] 
inte_2 = quad(f, 15.9, 16, args=(h, T, nj))[0]


0.8196106779364342 10.000706642322351


In [24]:
@jit(nopython=True)
def TD_1DChain(T, dmu, h, initial_chain=np.random.choice([-1,1], size=10000),
               P_gen_kj=np.ones((2,2))*0.5, J=4,
               num_reactions=10000000):
    '''
    Parameters
    ----------
    T: Period of the external cosine signal 
    dmu: The additional chemical potential from equilbirum 
    h:strength of the external signal
    
    P_gen_kj:  the probability of getting a particular particle nk from the bath given the interface particle is nj
       -1   1    
    -1 0.5 0.5
     1 0.5 0.5
    in general it is a matrix as above. Here just 0.5
    
    J: monomer-monomer interaction strength
    
    num_reactions: total reactions allowed
    initial_chain_size: size for the initial bulk
    
    Returns
    -------
    Timestamps and Changes of 1D-Chain
    '''
    #generate uniform random numbers  
    r_array = np.random.random(size=num_reactions)
    
    #track reaction time
    ln_inv_r = np.log(1/np.random.random(size=num_reactions))
    
    
    #Track the change of the 1D-Chain
    Chain_Changes = np.zeros(num_reactions+1, dtype=np.int8)
    #track time
    t = np.zeros(num_reactions+1, dtype=np.float32)
    t_rn = 0
    t_internal = 0 #internal time
    
    #track chain configuration
    chain = []
    chain.extend(initial_chain)
    
    #outmost blocks
    nj = initial_chain[-1]
    ni = initial_chain[-2]
    
    mu_eq = log(2/(exp(J)*(cosh(h)+sqrt(exp(-4*J)+sinh(h)**2))))
    mu = mu_eq+dmu
    exp_mu = exp(mu)
        
    #TODO: compute time-dependent propoensity for removal 
    exp_rem = exp(-J*ni*nj-h*np.cos(2*pi*t_rn/T)*nj)
    
    for i in range(num_reactions):
    #modified next reaction method
        
        t_d = ln_inv_
        
        #propensity for reactions
        if nj==-1:
            #waiting time for adding +1 to the chain
            tau_1 = exp_mu*P_gen_kj[0,1] 
            a2 = exp_mu*P_gen_kj[0,0]#adding -1 to the chain 
            a0 = a1+a2+exp_rem #total propensity
        else:
            a1 = exp_mu*P_gen_kj[1,1]#adding +1 to the chain 
            a2 = exp_mu*P_gen_kj[1,0]#adding -1 to the chain 
            a0 = a1+a2+exp_rem #total propensity
        
        #find and update arrival time
        r1 = r1_array[i] 
        tau = np.log(1/r1)/a0
        t_rn = t_rn+tau
        t[i] = t_rn
        
        # Threshold for selecting a reaction
        r2a0 = r2_array[i]*a0
        
        #choose a reaction
        #add +1
        if r2a0 < a1:
            Chain_Changes[i+1] = 1
            ni = nj
            nj = 1
            chain.append(1)
        #add -1
        elif a1 <= r2a0 <= a1+a2:
            Chain_Changes[i+1] = -1
            ni = nj
            nj = -1
            chain.append(-1)
        #removal
        else:
            Chain_Changes[i+1] = 0
            del chain[-1]
            nj = ni
            ni = chain[-1]
            
    return chain, Chain_Changes, t

In [25]:
#initial_chain = np.random.choice([-1,1], size=10000)

T = 1000
J = 4
h = 0.2
dmu = 0.3

chain, Chain_Changes, t = TD_1DChain(T, dmu, h)

print(t[-1])

0.0


In [ ]:
print(len(chain))